The unsupervised learning code of motorcycle crash analysis

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# Load unclean file
file = 'MotorBike_Accident_analysisUnclean.xlsx'
data = pd.read_excel(file)

In [3]:
# Clean names to keep only the English part of the column names
def clean_column(col):
	return col.split('(')[0].strip()

data.columns = [clean_column(col) for col in data.columns]

# Remove rest of strange languege from the rows
import re
def remove_parentheses(text):
	if isinstance(text, str):
		return re.sub(r'\s*\([^)]*\)', '', text).strip()
	return text

for col_idx, col in enumerate(data.columns):
	if col_idx != 5:  
		data[col] = data[col].apply(remove_parentheses)